<a href="https://colab.research.google.com/github/surjeetvermaiitm/Lear-Project/blob/master/OEE_Heat_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from pandas import DataFrame

%matplotlib inline
from IPython.display import display
from google.colab import drive

# Importing the file from Google drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/My Drive/Internship/HSL2_REPORT.csv")
n = df.shape[0]
df_OEE = df.head(n)

#Extracting required data from the data sheet
df_OEE = [[df_OEE['Date']],[df_OEE[' Time']],[df_OEE[' Position']],[df_OEE[' Line ID']],[df_OEE[' Result']] ] # Required matrix separarated
df_Time = df_OEE[1][0]  
df_Quality = df_OEE[4]

# Converting time to seconds
df_Time_Sec = np.empty([n,1])
for i in np.arange(0,n): 
  ftr = [3600,60,1]
  df_Time_Sec[i] = sum([a*b for a,b in zip(ftr, map(int,df_Time[i].split(':')))])

# Calculating time differences
df_Time_Diff = np.empty([n,1])
for i in np.arange(0,n):
  df_Time_Diff[i] = df_Time_Sec[i]-df_Time_Sec[i-1]
df_Time_Diff = df_Time_Diff[1:n]

# Extracting hour times from the Time data
df_Hour = []
df_Hourdata = df_Time_Sec/3600
df_Hourdata = df_Hourdata.astype(int)
for i in np.arange(n):
  if df_Hourdata[i] != df_Hourdata[i-1]:
    df_Hour.append(df_Hourdata[i])
df_Hour = np.asarray(df_Hour,'F')
df_Hour = df_Hour.astype(int)

# Generate a dummy array 
total_Hours = np.zeros(len(df_Hour))
total_Hours[0]= df_Hour[0]
for i in np.arange(1,len(total_Hours)):
  total_Hours[i] = total_Hours[i-1]+1
total_Hours = total_Hours.astype(int)

# Generating the Heat Map in numpy array format
Heat_Map_ = np.zeros([len(total_Hours),12])
k = 0
df_Time_Sec = np.append(df_Time_Sec,0 )
time_diff = 0
for i in df_Hour:
  for j in np.arange(12):
    m = ((i)*60*60+60*(j+1)*5)
    while df_Time_Sec[k] < m and k<n:
      if df_Quality[0][k] == 'OK':
        Heat_Map_[total_Hours[np.where(df_Hour==i)[0][0]]-total_Hours[0]][j] = Heat_Map_[total_Hours[np.where(df_Hour==i)[0][0]]-total_Hours[0]][j] + 1
      k = k+1
      time_diff = time_diff + df_Time_Sec[k]-df_Time_Sec[k-1]
      if(time_diff> 300 or time_diff<0):
        break
    time_diff = 0

# Converting data to a DataFrame and inserting Hour Column
df_Heat_Map = DataFrame(data = Heat_Map_.astype(int), columns = (['00:00'],['05:00'],['10:00'],['15:00'],['20:00'],['25:00'],['30:00'],['35:00'],['40:00'],['45:00'],['50:00'],['55:00']))
df_Heat_Map.insert(0,'\033'+"HOUR", df_Hour.astype(dtype = int))
df_Heat_Map = df_Heat_Map.rename_axis('Heat Map for every five minutes', axis=1) 
# Colouring the Data
def highlight_Number(number):
  color_list = []
  for i in number:
    if(i > 55):
      color_list.append('background-color: 	#008000')
    elif(i > 44 and i <= 55):
      color_list.append('background-color: 	#90EE90')
    elif(i>38 and i<45):
      color_list.append('background-color: 		#adff2f')
    elif(i<=38):
      color_list.append('background-color: 		#FF0000')
    else :
      color_list.append('')
  return color_list
Heat_Map = df_Heat_Map.style.apply(highlight_Number )
Heat_Map

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (229,230) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: ComplexWarning: Casting complex values to real discards the imaginary part


Heat Map for every five minutes,HOUR,"('00:00',)","('05:00',)","('10:00',)","('15:00',)","('20:00',)","('25:00',)","('30:00',)","('35:00',)","('40:00',)","('45:00',)","('50:00',)","('55:00',)"
0,7,18,43,43,35,30,51,45,46,38,44,55,45
1,8,28,37,29,36,53,33,26,7,49,20,6,20
2,9,6,0,8,51,50,52,31,31,26,55,35,6
3,10,25,47,44,30,44,54,56,48,16,55,23,28
4,11,0,0,0,0,0,0,25,38,51,40,25,45
5,12,11,0,14,50,12,6,0,32,59,44,39,24
6,13,0,5,18,35,41,38,0,2,42,0,14,35
7,14,45,12,2,0,5,24,25,18,23,3,6,31
8,15,17,48,25,21,13,20,0,20,19,28,31,31
9,16,33,45,31,12,47,44,12,38,31,17,45,53


In [153]:
k

6932